In [1]:
import khet
import random
import pylab as plt
import multiprocessing
%load_ext line_profiler
%load_ext memory_profiler

In [2]:
%%time
# Setup the game board
game_board = khet.engine.board.GameBoard('classic')

CPU times: user 74 µs, sys: 5 µs, total: 79 µs
Wall time: 83 µs


In [3]:
%%time
_ = khet.engine.evaluate_board_simple(game_board._board, 'red')

CPU times: user 24 µs, sys: 1 µs, total: 25 µs
Wall time: 29.1 µs


In [4]:
game_board.board_repr()

'Sdra0.Adre0.Pdrf0.Adrg0.prrh0.pdrc1.plsd2.pura3.pdsc3.Slre3.Surf3.prrh3.plsj3.prra4.plsc4.Srse4.Sdsf4.purh4.pdsj4.prrg5.push6.plsc7.Ausd7.Puse7.Ausf7.Susj7.'

In [5]:
%%time
ngames = 1000
for game in range(ngames):
    # Setup the game board
    game_board = khet.engine.board.GameBoard("classic")

    # Number of turns before stopping
    n_turns = 100
    nturn = 0
    is_game_active = True

    # Loop through turns
    #m_moves = []
    while is_game_active and nturn < n_turns:
        for color in ["silver", "red"]:

            options = game_board.get_all_valid_moves(color)
            piece_index = random.randint(0, len(options) - 1)
            
            piece, all_moves = options[piece_index]
            all_moves = list(all_moves)
            move_index = random.randint(0, len(all_moves) - 1)
            move, rotation = all_moves[move_index]
            #options = game_board.get_all_valid_moves(color)
            #piece_index = random.randint(0, len(options) - 1)
            #move_index = random.randint(0, len(options[piece_index][1]) - 1)
            
            #piece, moves = options[piece_index]
            #move, rotation = moves[move_index]
            #m_moves.append((piece.position, piece.__name__, move, rotation))

            # check
            #for ri, row in enumerate(game_board._board):
            #    for rj, item in enumerate(row):
            #        if item:
                        #print (game, nturn, item.__name__, item.position, (ri, rj), id(item))
            #            if item.position != (ri, rj):
                            #print (game, nturn, item.__name__, item.position, (ri, rj))
            #                raise ValueError
            
            # move piece
            #print (piece.position, move, rotation)
            #print("Turn start", piece.__name__, id(piece), 'move')
            #for ri, row in enumerate(game_board._board):
            #    for rj, item in enumerate(row):
            #        if item:
            #            print((item.__name__, item.position, (ri, rj), id(item), item.position == (ri, rj)))
            #print ()

            game_board.move_piece(
                piece.position, direction=move, rotation=rotation
            )
    
            #for ri, row in enumerate(game_board._board):
            #    for rj, item in enumerate(row):
            #        if item:
            #            print((item.__name__, item.position, (ri, rj), id(item)), item.position == (ri, rj))
            #print ()
            # End turn
            game_board.end_turn(color)

            is_game_active = game_board.is_game_over()

            nturn += 1

CPU times: user 3.03 s, sys: 15.3 ms, total: 3.05 s
Wall time: 3.06 s


In [12]:
def print_board(board):
    for row in board._board:
        row_name = ''
        for k in row:
            if k:
                if k.__name__ in ["Pyramid", "Sphinx"]:
                    row_name += k.__name__[:1].lower()
                else:
                    row_name += k.__name__[:1]
            else:
                row_name += "."

        print (row_name)

    print ()
    orientations = ["A", ">", "V", "<"]

    for row in board._board:
        row_name = ''
        for k in row:
            if k:
                row_name += orientations[k.orientation]
            else:
                row_name += "."

        print (row_name)

In [13]:
%%time
ngames = 1000
nturn_list = []
winners = []
for game in range(ngames):
    # Setup the game board
    game_board = khet.engine.board.GameBoard("classic")

    # Number of turns before stopping
    n_turns = 500
    nturn = 0
    pharaoh_exists = True

    # Loop through turns
    while pharaoh_exists and nturn < n_turns:
        for color in ["silver", "red"]:
            options = game_board.get_all_valid_moves(color)
            piece_index = random.randint(0, len(options) - 1)
            move_index = random.randint(0, len(options[piece_index][1]) - 1)
            
            piece, moves = options[piece_index]
            move, rotation = moves[move_index]

            # move piece
            game_board.move_piece(
                options[piece_index][0].position, direction=move, rotation=rotation
            )

            # End turn
            game_board.end_turn(color)

            #
            active_pieces = [
                piece
                for row in game_board._board
                for piece in row
                if piece and piece.__name__ == 'Pharaoh'
            ]
            
            if len(active_pieces) < 2:
                pharaoh_exists = False
                
                try:
                    if active_pieces[0].color == 'red':
                        winners.append('silver')
                    else:
                        winners.append('red')
                except:
                    winners.append(None)
                    
                break

            nturn += 1
            
    nturn_list.append(nturn)
    if nturn == n_turns:
        winners.append(None)

CPU times: user 3.01 s, sys: 15.4 ms, total: 3.03 s
Wall time: 3.04 s


In [15]:
def simulate_game(n_turns=100):
    board_state = []
    board_scores = []
    pharaoh_exists = True
    nturn = 0
    game_board = khet.engine.board.GameBoard("classic")
    while pharaoh_exists and nturn < n_turns:
        for color in ["silver", "red"]:
            options = game_board.get_all_valid_moves(color)
            piece_index = random.randint(0, len(options) - 1)
            move_index = random.randint(0, len(options[piece_index][1]) - 1)
            
            piece, moves = options[piece_index]
            move, rotation = moves[move_index]

            # move piece
            game_board.move_piece(
                options[piece_index][0].position, direction=move, rotation=rotation
            )

            # End turn
            game_board.end_turn(color)

            # Check if the game is over
            pharaoh_exists = game_board.is_game_over()
            
            board_state.append(game_board.board_repr())
            board_scores.append(khet.engine.evaluate_board_simple(game_board._board, color))
            nturn += 1
            
    return nturn, board_state, board_scores

In [19]:
%lprun -f khet.engine.GameBoard.get_all_valid_moves simulate_game()

Timer unit: 1e-09 s

Total time: 0.002068 s
File: /Users/tyler/mambaforge/envs/dev/lib/python3.10/site-packages/khet/engine/board.py
Function: get_all_valid_moves at line 310

Line #      Hits         Time  Per Hit   % Time  Line Contents
   310                                               def get_all_valid_moves(self, color) -> list:
   311                                                   """
   312                                           
   313                                                   Parameters:
   314                                                       color: str
   315                                                           Color of the player to get the valid moves for
   316                                                   """
   317         4       2000.0    500.0      0.1          all_moves = []
   318         8     175000.0  21875.0      8.5          active_pieces = [
   319                                                       piece
   320         4       